In [ ]:
!pip install torch torchvision
!pip install ultralytics -q
!pip install pyyaml -q
import torch
import pandas as pd
import time
from PIL import Image
import cv2
import os
import json
from ultralytics import YOLO
import yaml
from google.colab.patches import cv2_imshow


In [ ]:
file_name = "../usr/local/lib/python3.10/dist-packages/ultralytics/cfg/datasets/coco8.yaml"
with open (file_name, "r") as stream:
    names = yaml.safe_load(stream)["names"]

In [ ]:
coco_names= list(names.values())

In [ ]:



 
model = YOLO("yolov8n.pt")


if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available.")



In [ ]:

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip

import json

file_path = '/content/annotations/instances_val2017.json'

with open(file_path, 'r') as file:
    data = json.load(file)

file_name_to_id = {item['file_name']: item['id'] for item in data['images']}
print(file_name_to_id)

len(file_name_to_id)

if 'annotations' in data:
  
    category_ids = [annotation['category_id'] for annotation in data['annotations']]

    
    print("Sample category IDs:", category_ids[:10])  

    
    from collections import Counter
    category_id_counts = Counter(category_ids)
    print("Unique category IDs and their counts:", category_id_counts)

     
    print("Total unique category IDs:", len(category_id_counts))

else:
    print("The key 'annotations' does not exist in the data.")

 
 
 
 

 
if 'categories' in data:
     
    category_id_to_name = {category['id']: category['name'] for category in data['categories']}

     
    name_to_supercategory = {category['name']: category.get('supercategory', 'None') for category in data['categories']}

     
    print("Category ID to Name:", category_id_to_name)
    print("Name to Supercategory:", name_to_supercategory)
else:
    print("The key 'categories' does not exist in the data.")

category_id_to_name = {str(k): v for k, v in category_id_to_name.items()}
 
def convert_bbox_format(bbox):
    x_min, y_min, width, height = bbox
    x_max = x_min + width
    y_max = y_min + height
    return [x_min, y_min, x_max, y_max]

 
if 'annotations' in data:
    for annotation in data['annotations']:
         
        annotation['bbox_new'] = convert_bbox_format(annotation['bbox'])

         

else:
    print("The key 'annotations' does not exist in the data.")
 
if 'annotations' in data:
    for annotation in data['annotations']:
         
        category_id_str = str(annotation['category_id'])

         
        category_name = category_id_to_name.get(category_id_str, 'Unknown Category')

         
        annotation['category_name'] = category_name

         
        supercategory = name_to_supercategory.get(category_name, 'None')

         
        annotation['supercategory'] = supercategory

         
        print(f"Annotation ID: {annotation['bbox_new']}, Category Name: {category_name}, Supercategory: {supercategory}")

else:
    print("The key 'annotations' does not exist in the data.")

import pandas as pd

 
 

extracted_data = []
for item in data['annotations']:   
    extracted_data.append({
        'image id': item['image_id'],
        'name': item['category_name'],   
        'supercategory': item['supercategory'],
        'bounding box': item['bbox_new']
    })

 
df_annot = pd.DataFrame(extracted_data)

 
df_annot.reset_index(inplace=True, drop=True)
df_annot.sort_values(by='image id', ascending=True, inplace=True)
 

df_annot['gt_id'] = df_annot.index
df_annot['gt_id'] = df_annot['gt_id'].astype(int)

In [ ]:


times=[]
bboxes = []
scores = []
class_names = []
image_names = []
image_id_perdetection=[]
from os import listdir
directory="/content/val2017"
for image in os.listdir(directory):
  image_id = file_name_to_id[image]
  image_path = "/content/val2017/"+image


  start_time = time.time()

  results = model.predict(image_path, save_conf=True, conf =0.001)

  end_time = time.time()   

  elapsed_time = end_time - start_time
  times.append(elapsed_time)

   
  json_output = results[0].tojson()

 
  detections = json.loads(json_output)



   
  for detection in detections:
       
      box = detection["box"]
      bbox = [box["x1"], box["y1"], box["x2"], box["y2"]]
      bboxes.append(bbox)

       
      confidence = detection["confidence"]
      scores.append(confidence)

       
      class_name = detection["name"]
      class_names.append(class_name)

      image_id = file_name_to_id[image]
      image_id_perdetection.append(image_id)





In [ ]:


dict = {'image id': image_id_perdetection, 'name': class_names, 'bounding box': bboxes, 'confidence': scores }

df = pd.DataFrame(dict)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)   

df_sorted = df.sort_values(by='image id', ascending=True)

df_sorted.reset_index(inplace=True, drop=True)

 
df_sorted['bbox_id'] = df_sorted.index
df_sorted['bbox_id'] = df_sorted['bbox_id'].astype(int)


 


In [ ]:


 

def calculate_iou(boxA, boxB):
     
    xA, yA, wA, hA = boxA
    xB, yB, wB, hB = boxB
    areaA = wA * hA
    areaB = wB * hB

     
    xAmin, yAmin, xAmax, yAmax = xA, yA, xA + wA, yA + hA
    xBmin, yBmin, xBmax, yBmax = xB, yB, xB + wB, yB + hB

     
    xImin = max(xAmin, xBmin)
    yImin = max(yAmin, yBmin)
    xImax = min(xAmax, xBmax)
    yImax = min(yAmax, yBmax)

     
    interArea = max(xImax - xImin, 0) * max(yImax - yImin, 0)

     
    iou = interArea / float(areaA + areaB - interArea)
    return iou

 
df_sorted = df_sorted.sort_values(by='confidence', ascending=False)

 
matched_gt = set()
matched_preds = set()

matches = []

for _, pred_row in df_sorted.iterrows():
    pred_box = pred_row['bounding box']
    bbox_id = pred_row['bbox_id']
    image_id = pred_row['image id']
    class_name = pred_row['name']
    confidence = pred_row['confidence']

     
    if bbox_id in matched_preds:
        continue

     
    gt_filtered = df_annot[(df_annot['image id'] == image_id) &
                           (df_annot['name'] == class_name) &
                           (~df_annot['gt_id'].isin(matched_gt))]

    best_iou = 0
    best_match = None

    for _, gt_row in gt_filtered.iterrows():
        gt_box = gt_row['bounding box']
        gt_id = gt_row['gt_id']
        iou = calculate_iou(pred_box, gt_box)

        if iou > best_iou:
            best_iou = iou
            best_match = (gt_row['gt_id'], gt_box)

     
    if best_match and best_iou > 0.5:
        matched_gt.add(best_match[0])   
        matched_preds.add(bbox_id)   

        matches.append({
            'image id': image_id,
            'class name': class_name,
            'pred box': pred_box,
            'gt box': best_match[1],
            'iou': best_iou,
            'confidence': confidence,
            'bbox_id': bbox_id,
            'gt_id': best_match[0]
        })

 
df_matches_with_confidence = pd.DataFrame(matches)

 
 
 

 
pred_to_gt_mapping_issues = df_matches_with_confidence.duplicated(subset=['bbox_id'], keep=False)
if pred_to_gt_mapping_issues.any():
    print("Issues found with predicted bounding boxes being mapped to multiple ground truth boxes:")
    print(df_matches_with_confidence[pred_to_gt_mapping_issues].sort_values(by='bbox_id'))
else:
    print("All predicted bounding boxes are uniquely mapped to ground truth boxes.")

 
gt_to_pred_mapping_issues = df_matches_with_confidence.duplicated(subset=['gt_id'], keep=False)
if gt_to_pred_mapping_issues.any():
    print("\nIssues found with ground truth boxes being mapped to multiple predicted bounding boxes:")
    print(df_matches_with_confidence[gt_to_pred_mapping_issues].sort_values(by='gt_id'))
else:
    print("\nAll ground truth boxes are uniquely mapped to predicted bounding boxes.")



In [ ]:
 
matched_bbox_ids = set(df_matches_with_confidence['bbox_id'])

 
 
df_sorted['label'] = df_sorted['bbox_id'].apply(lambda x: 1 if x in matched_bbox_ids else 0)
from sklearn.metrics import average_precision_score

 

 
df_sorted = df_sorted.sort_values(by='confidence', ascending=False)

 
unique_classes = df_sorted['name'].unique()

 
ap_scores = []

 
for class_name in unique_classes:
     
    class_df = df_sorted[df_sorted['name'] == class_name]

     
    y_true = class_df['label'].tolist()   
    y_scores = class_df['confidence'].tolist()   

     
    ap = average_precision_score(y_true, y_scores)

     
    ap_scores.append({'class_name': class_name, 'AP': ap})

 
df_ap_scores = pd.DataFrame(ap_scores)
df_ap_scores

In [ ]:
 
filename = "class_ap_yoloagain.xlsx"

 
df_ap_scores.to_excel(filename, index=True)

from google.colab import files

 
files.download(filename)

In [ ]:
df_annot['bounding box'] = df_annot['bounding box'].apply(tuple)
df_matches_with_confidence['gt box'] = df_matches_with_confidence['gt box'].apply(tuple)

 
missing_boxes = ~df_annot['bounding box'].isin(df_matches_with_confidence['gt box'])

 
df_missing_boxes = df_annot[missing_boxes].reset_index(drop=True)
missing_boxes = ~df_annot['bounding box'].isin(df_matches_with_confidence['gt box'])

 
df_missing_boxes = df_annot[missing_boxes].reset_index(drop=True)

 
df_missing_boxes['area'] = df_missing_boxes['bounding box'].apply(lambda bbox: (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))

df_missing_boxes

In [ ]:


 

 
total_supercategory_counts = df_annot['supercategory'].value_counts()

 
missed_supercategory_counts = df_missing_boxes['supercategory'].value_counts()

 
 
missing_rate = (missed_supercategory_counts.reindex(total_supercategory_counts.index, fill_value=0)
                / total_supercategory_counts)

 
missing_rate_df = missing_rate.reset_index()
missing_rate_df.columns = ['Supercategory', 'Missing Rate']   

 
missing_rate_df.sort_values('Missing Rate', ascending=False, inplace=True)

 
missing_rate_df


In [ ]:
 

 
total_pixels = 640 * 640
small_threshold = total_pixels * 0.01   
medium_threshold = total_pixels * 0.1   

 
small_objects = 0
medium_objects = 0
large_objects = 0

 
for area in df_missing_boxes['area']:
    if area < small_threshold:
        small_objects += 1
    elif area < medium_threshold:
        medium_objects += 1
    else:
        large_objects += 1

 
print(f"Small objects: {small_objects}")
print(f"Medium objects: {medium_objects}")
print(f"Large objects: {large_objects}")
 
total_objects = medium_objects + small_objects + large_objects
ratio_of_small_to_rest = small_objects / total_objects

 
print("Ratio of small to rest: " + str(ratio_of_small_to_rest))



In [ ]:
df_tp = df_matches_with_confidence[df_matches_with_confidence['iou'] >= 0.5]

unique_classes = df_annot['name'].unique()
class_metrics = {}

for class_name in unique_classes:
     
    tp_class = df_tp[df_tp['class name'] == class_name]

     
    tp = len(tp_class)

     
    total_preds_for_class = len(df_sorted[df_sorted['name'] == class_name])
    fp = total_preds_for_class - tp

     
    total_gts_for_class = len(df_annot[df_annot['name'] == class_name])
    fn = total_gts_for_class - tp

     
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

     
    average_iou = tp_class['iou'].mean() if not tp_class.empty else 0

     
    class_metrics[class_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'Average IoU': average_iou
    }

 
df_class_metrics = pd.DataFrame.from_dict(class_metrics, orient='index')
df_class_metrics


In [ ]:
unique_class_names = df_annot['name'].unique().tolist()
count = 0
for i in unique_class_names:
  count = count +1

print(count)

In [ ]:
 
filename = "class_metrics_yolo.xlsx"

 
df_class_metrics.to_excel(filename, index=True)

from google.colab import files

 
files.download(filename)

In [ ]:
average_iou = df_matches_with_confidence['iou'].mean()

print("Average IoU:", average_iou)